<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_5/01_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 科研費概要の分類
日本語のデータセットでBERTのモデルをファインチューニングし、学術分野の分類を行います。  
『基盤BCで、2019年度から2021年度採択の４年分』  
"研究開始時の研究の概要"の文字列を使う。  
  
オリジナルの課題数： 48680  
概要が空白の課題数：   187  
空白を除いた課題数： 48493  
  
日本語＋英語： 48493  
英語　　　　：   646  
日本語　　　： 47847  

小区分がブランク：    54  
小区分の設定あり： 47793  
  
重複削除前の項目数： 323  
重複削除後の項目数： 315  
  
統合前のデータ数： 47793  
統合したデータ数： 49036  
トレーニングデータ数： 36777  
テストデータ数　　　： 12259  

### ライブラリのインストール
SageMakerでは初回だけで良い（Google colaboratoryでは毎回）  
ライブラリTransformers、およびnlpをインストールします。  

In [ ]:
# pipのアップデート（たまに走らせても良いかも）
!pip list
!python -m pip install --upgrade pip

In [ ]:
# ライブラリのインストール
!pip install torch
!pip install transformers
!pip install nlp
!pip install datasets
!pip install fugashi
!pip install ipadic
!pip install scikit-learn
!pip install matplotlib
!pip install tensorboard

### Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
# SageMakerでは不要
# from google.colab import drive
# drive.mount("/content/drive/")

## ファインチューニング用データの読み込み＆書き出し
### 科研費データの読み込み、整理
科研費データベースからダウンロードしたcsvファイルを直接読む  
必要なデータを取り出す。

In [ ]:


import pandas as pd

# 科研費データベースからダウンロードした未加工のcsvファイルを指定
# 基盤BC　2019年度～2021年度の４年分（2018年度は”研究開始時の研究の概要”が無い。2022年度は後半の検証に用いる）
open_original_csv = "KibanBC_2019-2021.csv"

# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "../data/" # SageMaker 

# csvファイルを開く
raw_data = pd.read_csv(data_path + open_original_csv) # dtype="object"必要？

# 読み込んだデータをチェック
# raw_data.info()

# 今後必要な行だけを取り出し、リネーム
# kadai = raw_data[["研究課題/領域番号", "審査区分", "研究課題名", "研究開始時の研究の概要", "研究成果の概要", "研究実績の概要", "キーワード"]]
# kadai.columns = ["ID", "ShoKubun", "Title", "Abst1", "Abst2", "Abst3", "Keyword"]
kadai = raw_data[["研究課題/領域番号", "審査区分", "研究開始時の研究の概要"]]
kadai.columns = ["ID", "ShoKubun", "Abst"]

# 課題番号の重複を確認。課題番号でソートする。
kadai["ID"].duplicated().any()
# kadai = kadai.set_index("ID") # IDをインデックスに設定するコード
kadai = kadai.sort_values("ID")
kadai.reset_index(inplace=True, drop=True)

# Abstの各項目が英語だけの場合、ブランクに入れ替える
# kadai.Abst1[~kadai["Title"].str.contains(r'[ぁ-んァ-ン]', na=True)] = ""
# kadai.Abst2[~kadai["Abst1"].str.contains(r'[ぁ-んァ-ン]', na=True)] = ""
# kadai.Abst3[~kadai["Abst2"].str.contains(r'[ぁ-んァ-ン]', na=True)] = ""
# kadai.Abst1[~kadai["Abst3"].str.contains(r'[ぁ-んァ-ン]', na=True)] = ""

# 研究開始時の研究の概要、研究成果の概要、研究実績の概要、キーワードを結合
# kadai['Abst'] = kadai['Title'].fillna('') + kadai['Abst1'].fillna('') + kadai['Abst2'].fillna('') + kadai['Abst3'].fillna('') + kadai['Keyword'] # t01
# kadai['Abst'] = kadai['Title'].fillna('') + '。' + kadai['Keyword'] + '。' + kadai['Abst1'].fillna('') + kadai['Abst2'].fillna('') + kadai['Abst3'].fillna('') # t02



# Abstが空欄の課題を削除
print("オリジナルの課題数： %5d" % len(kadai))
print("概要が空白の課題数： %5d" % len(kadai[kadai["Abst"].isna()]))
kadai = kadai.dropna(subset=["Abst"])
print("空白を除いた課題数： %5d\n" % len(kadai))

# Abst中の改行コード、全角スペースを削除
kadai = kadai.replace('\r', '', regex=True) # Carriage Return(MacOS9) \r\n for Windows
kadai = kadai.replace('\n', '', regex=True) # Line Feed（Unix MacOSX）
# kadai = kadai.replace(' ', '', regex=True) # 半角スペースは英語があるので削除しない
kadai = kadai.replace('　', '', regex=True) # 全角スペース

# Abstが英語のみの課題を削除
num_jpen = len(kadai)
kadai = kadai[kadai["Abst"].str.contains(r'[ぁ-んァ-ン]')]
num_jp   = len(kadai)
print("日本語＋英語： %5d" % num_jpen)
print("英語　　　　： %5d" % (num_jpen - num_jp))
print("日本語　　　： %5d\n" % num_jp)

# 小区分が設定されていない課題を削除（旧分類、特設分野）
aaa = len(kadai)
kadai = kadai.dropna(subset=["ShoKubun"])
print("小区分がブランク： %5d" % (aaa - len(kadai)))
print("小区分の設定あり： %5d\n" % len(kadai))

# 小区分の文字列の数字部分だけを取り出す
kadai["ShoKubun"] = kadai["ShoKubun"].str[3:8]
kadai = kadai.astype({"ShoKubun": int})

kadai

### 整理した科研費データの保存
審査区分データを読み込み、小区分番号を参照して結合  
トレーニングデータと、テストデータに分けて保存する。

In [ ]:
from sklearn.model_selection import train_test_split

# 科研費の審査区分表データのcsvファイル
open_kubun_csv = "KubunTable.csv"
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker

# 書き出し用CSVファイル名
train_csv = data_path + "kadai_train.csv"
test_csv  = data_path + "kadai_test.csv"

# ============================================================================

# 審査区分テーブルのロード
kubun_table = pd.read_csv(data_path + open_kubun_csv, encoding="cp932")
kubun_table = kubun_table[["tabDai", "tabSho"]]

# 審査区分表の重複を削除（一つの小区分が２つまたは３つの『中区分』に所属することに由来する）
print("重複削除前の項目数： %3d" % len(kubun_table))
kubun_table = kubun_table.drop_duplicates()
print("重複削除後の項目数： %3d\n" % len(kubun_table))

## 中区分への変換
## mergeを用いて、審査区分表のデータと突合
#print("統合前のデータ数： %5d" % len(kadai))
#kadaiChu = pd.merge(kadai, kubun_table, left_on='ShoKubun', right_on='tabSho')
#kadaiChu = kadaiChu[["Abst", "tabChu", "ID", "ShoKubun"]]
#print("統合したデータ数： %5d\n" % len(kadaiChu))

# 大区分への変換
# mergeを用いて、審査区分表のデータと突合
print("統合前のデータ数： %5d" % len(kadai))
kadaiDai = pd.merge(kadai, kubun_table, left_on='ShoKubun', right_on='tabSho')
kadaiDai = kadaiDai[["Abst", "tabDai", "ID", "ShoKubun"]]
print("統合したデータ数： %5d" % len(kadaiDai))

# 訓練用とテスト用に分割 層化
kadai_train, kadai_test =  train_test_split(kadaiDai, shuffle=True, stratify = kadaiDai["tabDai"].tolist())
print("トレーニングデータ数： %5d"   % len(kadai_train))
print("テストデータ数　　　： %5d\n" % len(kadai_test))



## 訓練用とテスト用に分割 層化
#kadai_train, kadai_test =  train_test_split(kadaiChu, shuffle=True, stratify = kadaiChu["tabChu"].tolist())
#print("トレーニングデータ数： %5d" % len(kadai_train))
#print("テストデータ数　　　： %5d\n" % len(kadai_test))

# 課題番号→小区分→中区分を基準にソート（計算的には不要だが、人間用にソートしておく）
kadai_train = kadai_train.sort_values(["tabDai", "ShoKubun", "ID"])
kadai_test  = kadai_test.sort_values (["tabDai", "ShoKubun", "ID"])

# ソート用に残していた課題番号（ID）行を削除
#kadai_train = kadai_train.drop(['ID'], axis=1)
#kadai_test  = kadai_test.drop (['ID'], axis=1)
kadai_train = kadai_train.drop(['ID', 'ShoKubun'], axis=1)
kadai_test  = kadai_test.drop (['ID', 'ShoKubun'], axis=1)

# csvとして書き出し
kadai_train.to_csv(train_csv, header=False, index=False)
kadai_test.to_csv (test_csv,  header=False, index=False)

print("Saved\n %s\n %s\n" % (train_csv, test_csv))

## ファインチューニングの実施
### モデルとTokenizerの読み込み
日本語の事前学習済みモデルと、これと紐づいたTokenizerを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

sc_model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", num_labels=11) # 大区分は11
sc_model.cuda() # GPUを使う
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

print("Finish")

### データセットの読み込み
さきほど作成した科研費データ（training,test）を読み込みます。

In [ ]:
from datasets import load_dataset

def tokenize(batch):
    # return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)
    
data_path = "./"

train_data = load_dataset("csv", data_files=data_path+"news_train.csv", column_names=["text", "label"], split="train")
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])

test_data = load_dataset("csv", data_files=data_path+"news_test.csv", column_names=["text", "label"], split="train")
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))
test_data.set_format("torch", columns=["input_ids", "label"])

print("Finish")

## 評価用の関数
`sklearn.metrics`を使用し、モデルを評価するための関数を定義します。  


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

print("Finish")

### Trainerの設定
Trainerクラス、およびTrainingArgumentsクラスを使用して、訓練を行うTrainerの設定を行います。  
https://huggingface.co/transformers/main_classes/trainer.html  
https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments  

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 2,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,  # 学習係数が0からこのステップ数で上昇
    weight_decay = 0.01,  # 重みの減衰率
    # evaluate_during_training = True,  # ここの記述はバージョンによっては必要ありません
    logging_dir = "./logs",
    save_total_limit=1,  # limit the total amount of checkpoints. Deletes the older checkpoints.
)

trainer = Trainer(
    model = sc_model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = test_data,
)

print("Finish")

### モデルの訓練
設定に基づきファインチューニングを行います。  
40分程度かかる。 colaboratory 32分かかった　sagemaker

In [ ]:
trainer.train()

print("Finish")

### モデルの評価
Trainerの`evaluate()`メソッドによりモデルを評価します。  
２分程度かかる

In [ ]:
trainer.evaluate()

print("Finish")

### TensorBoardによる結果の表示
TensorBoardを使って、logsフォルダに格納された学習過程を表示します。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

### モデルの保存
訓練済みのモデルを保存します。

In [ ]:
# data_path = "/content/drive/My Drive/bert_nlp/section_5/"
data_path = "./"

sc_model.save_pretrained(data_path)
tokenizer.save_pretrained(data_path)

print("Finish")

# 以降は削除予定

## モデルの読み込み
保存済みのモデルを読み込みます。

In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained(data_path) 
loaded_model.cuda()
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(data_path)

## 日本語ニュースの分類
読み込んだモデルを使ってニュースを分類します。

In [ ]:
import glob  # ファイルの取得に使用
import os
import torch

category = "movie-enter"
sample_path = "/content/drive/My Drive/bert_nlp/section_5/text/"  # フォルダの場所を指定
files = glob.glob(sample_path + category + "/*.txt")  # ファイルの一覧
file = files[12]  # 適当なニュース

dir_files = os.listdir(path=sample_path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(sample_path, f))]  # ディレクトリ一覧

with open(file, "r") as f:
    sample_text = f.readlines()[3:]
    sample_text = "".join(sample_text)
    sample_text = sample_text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""})) 

print(sample_text)

max_length = 512
words = loaded_tokenizer.tokenize(sample_text)
word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
word_tensor = torch.tensor([word_ids[:max_length]])  # テンソルに変換

x = word_tensor.cuda()  # GPU対応
y = loaded_model(x)  # 予測
pred = y[0].argmax(-1)  # 最大値のインデックス
print("result:", dirs[pred])